# Question Answering with Similarity Search

This notebook demonstrates how Pinecone's similarity search as a service helps you build a question answering application. We will index a set of questions and retrieve the most similar stored questions for a new (unseen) question. That way, we can link a new question to answers we might already have.

You can build a questions answering application with Pinecone in three steps:
- Represent questions as vector embeddings so that semantically similar questions are in close proximity within the same vector space. 
- Index vectors using Pinecone.
- Given a new question, query the index to fetch similar questions. This can allow us to store answers associated with these questions 

In this notebook we will be dealing with indexing a set of quetions and retrieving similar questions for a new and unseen question.



## Dependencies

In [1]:
!pip install -qU matplotlib pinecone-client ipywidgets
!pip install -qU sentence-transformers --no-cache-dir

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

## Pinecone Installation and Setup

In [28]:
import pinecone
import os

# load Pinecone API key
api_key = os.getenv("PINECONE_API_KEY") or "YOUR_API_KEY"
pinecone.init(api_key=api_key, environment='us-west1-gcp')

[Get a Pinecone API key](https://www.pinecone.io/start/) if you don’t have one already.

## Create a New Pinecone Index

In [3]:
# pick a name for the new index
index_name = "question-answering"

In [4]:
# check whether an index with the same name already exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

**Create index**

In [7]:
pinecone.create_index(name=index_name, dimension=300, metric="cosine", shards=1)

**Connect to the index**

The index object, a class instance of pinecone.Index , will be reused for optimal performance.

In [29]:
index = pinecone.Index(index_name=index_name)

## Uploading Questions

The dataset used in this notebook is the [Quora Question Pairs Dataset](https://www.kaggle.com/c/quora-question-pairs).

Let's download the dataset and load the data.

In [5]:
# download dataset from the url
import requests

DATA_DIR = "tmp"
DATA_FILE = f"{DATA_DIR}/quora_duplicate_questions.tsv"
DATA_URL = "https://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"


def download_data():
    os.makedirs(DATA_DIR, exist_ok=True)

    if not os.path.exists(DATA_FILE):
        r = requests.get(DATA_URL)  # create HTTP response object
        with open(DATA_FILE, "wb") as f:
            f.write(r.content)


download_data()

In [16]:
pd.set_option("display.max_colwidth", 500)

df = pd.read_csv(
    f"{DATA_FILE}", sep="\t", usecols=["qid1", "question1"], index_col=False
)
df = df.sample(frac=1).reset_index(drop=True)
df.drop_duplicates(inplace=True)
df['qid1'] = df['qid1'].apply(str)
print(df.head())

     qid1  \
0   59978   
1  319800   
2   59731   
3  226004   
4  228205   

                                                                                                                                      question1  
0                                                        What is a suitable solar panel installation provider near San Clemente, California CA?  
1                                                                                                                Hollywood MOVIE DOWNLOAD SITE?  
2  If the message I sent to someone on WhatsApp is later than her last seen, how could the message be seen, as it is shown by double tick sign?  
3                                                                                                            Why do I hardly feel any emotions?  
4                                                                                              What is the best combination within a CA course?  


### Define the model

We will use the [Averarage Word Embeddings Model](https://nlp.stanford.edu/projects/glove/) for this example. This model has a high computation speed but relatively low quality of embeddings. You can look into other sentence embeddings models such as the [Sentence Embeddings Models trained on Paraphrases](https://huggingface.co/sentence-transformers/paraphrase-distilroberta-base-v1) for improving quality of embeddings. 

In [17]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("average_word_embeddings_glove.6B.300d")

### Creating Vector Embeddings

In [18]:
# create embedding for each question
df["question_vector"] = df.question1.apply(lambda x: model.encode(str(x)).tolist())

### Index the Vectors

In [19]:
import itertools

def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [ ]:
for batch in chunks(zip(df.qid1, df.question_vector)):
    index.upsert(vectors=batch)

## Search

Once you have indexed the vectors it is very straightforward to query the index. These are the steps you need to follow:
- Select a set of questions you want to query with
- Use the Average Embedding Model to transform questions into embeddings.
- Send each question vector to the Pinecone index and retrieve most similar indexed questions

In [26]:
# define questions to query the vector index
query_questions = [
    "What is best way to make money online?",
]

# extract embeddings for the questions
query_vectors = [model.encode(str(question)).tolist() for question in query_questions]

# query pinecone
query_results = index.query(queries=query_vectors, top_k=5)

# show the results
for question, res in zip(query_questions, query_results.results):
    print("\n\n\n Original question : " + str(question))
    print("\n Most similar questions based on pinecone vector search: \n")

    ids = [match.id for match in res.matches]
    scores = [match.score for match in res.matches]
    df_result = pd.DataFrame(
        {
            "id": ids,
            "question": [
                df[df.qid1 == _id].question1.values[0] for _id in ids
            ],
            "score": scores,
        }
    )
    print(df_result)




 Original question : What is best way to make money online?

 Most similar questions based on pinecone vector search: 

       id                                       question     score
0  297469     What is the best way to make money online?  1.000000
1      57         What is best way to make money online?  1.000000
2   55585  What is the best way for making money online?  0.989930
3   28280   What are the best ways to make money online?  0.981526
4    7800            What is a way to make money online?  0.975910


## Delete the Index

Delete the index once you are sure that you do not want to use it anymore. Once it is deleted, you cannot reuse it.


In [31]:
pinecone.delete_index(index_name)